In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlproject/label_encoder.pkl
/kaggle/input/mlproject/2_climate_analysis.png
/kaggle/input/mlproject/1_crop_distribution.png
/kaggle/input/mlproject/__results__.html
/kaggle/input/mlproject/3_soil_analysis.png
/kaggle/input/mlproject/4_model_comparison.png
/kaggle/input/mlproject/crop_model.pkl
/kaggle/input/mlproject/scaler.pkl
/kaggle/input/mlproject/__notebook__.ipynb
/kaggle/input/mlproject/__output__.json
/kaggle/input/mlproject/custom.css
/kaggle/input/mlproject/__results___files/__results___9_1.png
/kaggle/input/mlproject/__results___files/__results___5_1.png
/kaggle/input/mlproject/__results___files/__results___3_0.png
/kaggle/input/mlproject/__results___files/__results___4_1.png


In [2]:
import joblib
import requests
import numpy as np
import os

In [3]:
MODEL_PATH = '/kaggle/input/mlproject/'

In [4]:
YOUR_SOIL_DATA = {
    "N": 90,  
    "P": 45,  
    "K": 45,  
    "ph": 6.5 
}

LATITUDE = 13.08
LONGITUDE = 80.27
print(f"Loading files from: {MODEL_PATH}")
if not os.path.exists(MODEL_PATH):
    print(f"ERROR: Cannot find path. Did you add your Part 1 notebook as data?")
    print("Please check the 'MODEL_PATH' variable and your notebook's input data.")
else:
    print("Found files:")
    print(os.listdir(MODEL_PATH))
try:
    model = joblib.load(MODEL_PATH + 'crop_model.pkl')
    scaler = joblib.load(MODEL_PATH + 'scaler.pkl')
    encoder = joblib.load(MODEL_PATH + 'label_encoder.pkl')
    print("\nModel, Scaler, and Encoder loaded successfully. 🤖")
except Exception as e:
    print(f"\nError loading .pkl files: {e}")
    print("Please ensure 'crop_model.pkl', 'scaler.pkl', and 'label_encoder.pkl' are in the output of your Part 1 notebook.")

Loading files from: /kaggle/input/mlproject/
Found files:
['label_encoder.pkl', '2_climate_analysis.png', '1_crop_distribution.png', '__results__.html', '3_soil_analysis.png', '4_model_comparison.png', 'crop_model.pkl', 'scaler.pkl', '__notebook__.ipynb', '__results___files', '__output__.json', 'custom.css']

Model, Scaler, and Encoder loaded successfully. 🤖


In [7]:
def get_weather_data(lat, lon):
    print(f"\nFetching live weather for {lat}, {lon}... 🌦️")
    try:
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": lat,
            "longitude": lon,
            "current": ["temperature_2m", "relative_humidity_2m", "precipitation"],
            "daily": "precipitation_sum",
            "timezone": "auto"
        }
        response = requests.get(url, params=params)
        response.raise_for_status() 
        data = response.json()
        
        current_weather = data['current']
        daily_weather = data['daily']
        
        weather_report = {
            "temperature": current_weather['temperature_2m'],
            "humidity": current_weather['relative_humidity_2m'],
            "current_rainfall": current_weather['precipitation'],
            "today_total_rainfall": daily_weather['precipitation_sum'][0]
        }
        print(f"Weather Report: {weather_report}")
        return weather_report
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None

In [9]:
def get_crop_recommendation(N, P, K, temp, humidity, ph, rainfall):
    
    user_input = np.array([[N, P, K, temp, humidity, ph, rainfall]])
    
    user_input_scaled = scaler.transform(user_input)
    
    prediction_encoded = model.predict(user_input_scaled)
    
    crop_name = encoder.inverse_transform(prediction_encoded)
    
    return crop_name[0]

In [11]:
def should_irrigate(weather_data, recommended_crop):
    if weather_data is None:
        return "Cannot decide: Weather data unavailable."
    
    temp = weather_data['temperature']
    humidity = weather_data['humidity']
    total_rain = weather_data['today_total_rainfall']
    
    print(f"\n--- Irrigation Logic ---")
    print(f"Recommended Crop: {recommended_crop}")
    print(f"Today's Total Rainfall: {total_rain} mm")
    print(f"Current Humidity: {humidity}%")
    
    
    if total_rain > 5:
        return "Decision: DO NOT IRRIGATE. (Sufficient rainfall today)"

   
    if recommended_crop == 'rice':
        if total_rain < 2 and humidity < 80:
            return "Decision: IRRIGATE. (Crop 'rice' needs water, low rain/humidity)"
    
    elif recommended_crop in ['grapes', 'pomegranate', 'chickpea']:
        if total_rain < 1 and humidity < 40 and temp > 28:
            return "Decision: IRRIGATE. (Tolerant crop, but it's very hot/dry)"

    if total_rain == 0 and humidity < 50:
        return "Decision: IRRIGATE. (Conditions are very dry)"

    return "Decision: DO NOT IRRIGATE. (Conditions are moderate)"

In [16]:
def run_app():
    weather = get_weather_data(LATITUDE, LONGITUDE)
    
    if weather:
        crop = get_crop_recommendation(
            YOUR_SOIL_DATA["N"],
            YOUR_SOIL_DATA["P"],
            YOUR_SOIL_DATA["K"],
            weather['temperature'],
            weather['humidity'],
            YOUR_SOIL_DATA["ph"],
            weather['today_total_rainfall'] 
        )
        
        print(f"\n--- Crop Recommendation ---")
        print(f"Based on soil and live weather, best crop is: {crop.upper()}")
        
        decision = should_irrigate(weather, crop)
        
        print("\n" + "="*30)
        print(f"FINAL DECISION: {decision}")
        print("="*30)
        
        if "IRRIGATE" in decision:
            print("\nAction: (Simulation) Sending signal to irrigation pump... 💧")
        else:
            print("\nAction: (Simulation) No irrigation needed.")

In [17]:
run_app()


Fetching live weather for 13.08, 80.27... 🌦️
Weather Report: {'temperature': 29.0, 'humidity': 79, 'current_rainfall': 0.0, 'today_total_rainfall': 0.2}

--- Crop Recommendation ---
Based on soil and live weather, best crop is: MUSKMELON

--- Irrigation Logic ---
Recommended Crop: muskmelon
Today's Total Rainfall: 0.2 mm
Current Humidity: 79%

FINAL DECISION: Decision: DO NOT IRRIGATE. (Conditions are moderate)

Action: (Simulation) Sending signal to irrigation pump... 💧


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
